#### Лабораторная работа 1. Отсекающие ограничения. Метод Гомори
Реализовать алгоритм Гомори для решения задачи целочисленного линейного программирования методом отсекающих ограничений.

$----------------------------------------------$

In [179]:
import numpy as np
from scipy.optimize import linprog

<b>Шаг 1.</b> В задаче целочисленного линейного программирования ($\mathnormal{IP}$) отбрасываем условия целочисленности переменных и получим задачу линейного программирования ($\mathnormal{LP}$).

In [180]:
A = np.array([  [3, 2, 1, 0],
                [-3, 2, 0, 1]   ])
b = np.array([6, 0])
c = np.array([0, 1, 0, 0])
c = -c

<b>Шаг 2.</b> Решаем задачу ($\mathnormal{LP}$) симплекс-методом.

In [181]:
simplex_result = linprog(c, A_ub=A, b_ub=b, method='highs')

<b>Шаг 3.</b> Если ($\mathnormal{LP}$) несовместена или ее целевая функция неограничена сверху на множестве допустимых планов, то метод заканчивает работу. Иначе продолжаем.

In [182]:
if simplex_result.success:
    print ("Задача несовместена или ее целевая функция неограничена сверху на множестве допустимых планов")

Задача несовместена или ее целевая функция неограничена сверху на множестве допустимых планов


<b>Шаг 4.</b> Пусть симплекс-метод на <b>Шаг 2</b> нашел оптимальный базисный допустимый план $\overline{x}$ с множеством базисных индексов $\mathnormal{B}$. Тогда, проверим следующее:

$$
\overline{x} \in \mathbb{Z}^{n}
$$

In [183]:
is_over = all(np.isclose(x, np.round(x)) for x in simplex_result.x)

Если условие верно, то $\overline{x}$ — оптимальный план ($\mathnormal{IP}$), метод заканчивает свою работу

In [184]:
if is_over:
    print(f"Оптимальный план: {simplex_result.x}")

Иначе добавим отсекающее ограничение Гомори к задаче ($\mathnormal{IP}$) и переходим на <b>Шаг 2</b>.

Найдем в $\mathnormal{B}$ индекс $\mathnormal{k}$ элемента из $\overline{x}$ с самой большой дробной частью

In [185]:
B = [i for i in range(len(simplex_result.x)) if simplex_result.x[i] != 0]
x_frac = simplex_result.x - np.floor(simplex_result.x)
k = 0
for i in B:
    if x_frac[k] < x_frac[i]:
        k = i
k

1

Соберем матрицы $\mathnormal{A}_{B}$ и $\mathnormal{A}_{N}$, где $\mathnormal{A}_{B}$ состоит из столбцов матрицы $\mathnormal{A}$, чьи индексы есть в $\mathnormal{B}$, а $\mathnormal{A}_{N}$ из всех остальных

In [186]:
A_B = A[:, B]
A_N = A[:, [i for i in range(A.shape[1]) if i not in B]]

print(f"A_B = {A_B}, \n\nA_N = {A_N}")

A_B = [[ 3  2]
 [-3  2]], 

A_N = [[1 0]
 [0 1]]


Найдем матрицу ${\mathnormal{A}_{B}}^{-1}$

In [187]:
A_B_inv = np.linalg.inv(A_B) 
A_B_inv

array([[ 0.16666667, -0.16666667],
       [ 0.25      ,  0.25      ]])

Для получения вектора $\mathnormal{l}$ необходимо перемножить матрицы ${\mathnormal{A}_{B}}^{-1}$ и $\mathnormal{A}_{N}$, после чего взять $\mathnormal{k}$-ю строку из результата 

In [188]:
l = np.dot(A_B_inv, A_N)[k, :]
l

array([0.25, 0.25])

Данные коэффициенты будут использованы для получения отсекающего ограничения. Делается это в два простых шага:

<b>Шаг 1.</b> Составим неравенство Гомори, вида
$$
\{\mathnormal{x}_{\mathnormal{k}}\} - \sum_{i=1}^{n}l_{i} * {\overline{x}'}_i \leq 0
$$
где ${\overline{x}'}$ множество всех небазисных элементов $\overline{x}$, а n — их количество

В нашем случае получим
$$
0.5 - \frac{1}{4}x_{3} - \frac{1}{4}x_{4} \leq 0
$$

Перенесем известные значения в правую часть

$$
- \frac{1}{4}x_{3} - \frac{1}{4}x_{4} \leq -0.5
$$

Добавим новую переменную для уравнивания левой и правой части неравенства

$$
- \frac{1}{4}x_{3} - \frac{1}{4}x_{4} + x_{5} = -0.5
$$

И домножим обе части на $-1$

$$
\frac{1}{4}x_{3} + \frac{1}{4}x_{4} - x_{5} = 0.5
$$

Итоговое ограничение

$$
\frac{1}{4}x_{3} + \frac{1}{4}x_{4} - x_{5} = 0.5
$$